In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# import plotly.express as px
# from pandas_profiling import ProfileReport
# from sklearn.impute import KNNImputer
import seaborn as sns
data = pd.read_csv('./data/res_clean.csv', low_memory=False)


pd.options.display.max_columns = None

FileNotFoundError: [Errno 2] No such file or directory: './data/res_clean.csv'

In [ ]:
df = 